In [1]:
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
import torch
device = 0 if torch.cuda.is_available() else -1  # 0 = first GPU, -1 = CPU for speed
from arabert.preprocess import ArabertPreprocessor #preprocess text
from dotenv import load_dotenv
import re
from nltk.tokenize import sent_tokenize
import random
from transformers import pipeline


load_dotenv()  # Loads HF_TOKEN automatically

C:\Users\renad\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
#directly thro hugging face
from transformers import pipeline

text = "إمارة أبوظبي هي إحدى إمارات دولة الإمارات العربية المتحدة السبع"

# POS tagging
pos = pipeline(
    "token-classification",
    model="CAMeL-Lab/bert-base-arabic-camelbert-da-pos-msa",
    framework="pt",
    trust_remote_code=True  # VERY IMPORTANT
)

# NER
ner = pipeline(
    "ner",
    model="CAMeL-Lab/bert-base-arabic-camelbert-msa-ner",
    framework="pt",
    trust_remote_code=True  # VERY IMPORTANT
)

print("POS:", pos(text))
print("NER:", ner(text))


Some weights of the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-da-pos-msa were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
Some weights of the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-msa-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoin

POS: [{'entity': 'noun', 'score': 0.9999914, 'index': 1, 'word': 'إمارة', 'start': 0, 'end': 5}, {'entity': 'noun_prop', 'score': 0.99924755, 'index': 2, 'word': 'أبوظبي', 'start': 6, 'end': 12}, {'entity': 'pron', 'score': 0.9999192, 'index': 3, 'word': 'هي', 'start': 13, 'end': 15}, {'entity': 'noun', 'score': 0.9999318, 'index': 4, 'word': 'إحدى', 'start': 16, 'end': 20}, {'entity': 'noun', 'score': 0.99999106, 'index': 5, 'word': 'إما', 'start': 21, 'end': 24}, {'entity': 'noun', 'score': 0.99998987, 'index': 6, 'word': '##رات', 'start': 24, 'end': 27}, {'entity': 'noun', 'score': 0.99999344, 'index': 7, 'word': 'دولة', 'start': 28, 'end': 32}, {'entity': 'noun', 'score': 0.99998987, 'index': 8, 'word': 'الإمارات', 'start': 33, 'end': 41}, {'entity': 'adj', 'score': 0.9999056, 'index': 9, 'word': 'العربية', 'start': 42, 'end': 49}, {'entity': 'adj', 'score': 0.9999796, 'index': 10, 'word': 'المتحدة', 'start': 50, 'end': 57}, {'entity': 'noun_num', 'score': 0.9993895, 'index': 11, '

In [19]:
def generate_question_text(entity, pos, word, sentence=None):
    """Generate natural Arabic question from entity + POS MQ."""
    
    # Person + adjective
    if entity == "PER" and pos == "adj":
        return f"من لديه {word}؟"
    
    # Person + proper noun
    if entity == "PER" and pos == "noun_prop":
        return f"من هو {word}؟"
    
    # Location + any noun
    if entity == "LOC":
        return f"أين يقع {word}؟"
    
    # Organization + adjective
    if entity == "ORG" and pos == "adj":
        return f"ما هي المنظمة التي وُصفت بأنها {word}؟"
    
    # Cloze/fill-in-the-blank
    if sentence:
        return f"أكمل الفراغ: {sentence.replace(word, '____')}"
    
    return f"صف الكلمة: {word}"


In [20]:
import random

def generate_TF_question(sentence, entity_word, entity_type, ner_tags):
    """
    Generate a True/False question by either keeping the entity (True)
    or replacing it with a realistic distractor of the same entity type (False).
    
    sentence: the original sentence
    entity_word: the correct entity in the sentence
    entity_type: NER type, e.g., "LOC", "PER", "ORG"
    ner_tags: output from ner() for the text

    the negiation doesnt seem realistic
    """
    
    # True statement
    true_statement = f"{sentence} (صح أم خطأ؟)"
    
    # Find other entities of the same type as distractors
    other_entities = [tag['word'] for tag in ner_tags 
                      if tag['entity'] == entity_type and tag['word'] != entity_word]
    
    # Pick one as distractor if exists, otherwise fallback
    distractor = random.choice(other_entities) if other_entities else entity_word + "X"
    
    # False statement: replace entity with distractor
    false_statement = sentence.replace(entity_word, distractor) + " (صح أم خطأ؟)"
    
    # Randomly choose True or False
    if random.choice([True, False]):
        return {"type": "TF", "statement": true_statement, "answer": True}
    else:
        return {"type": "TF", "statement": false_statement, "answer": False}


In [21]:
def generate_MCQ_question(entity_word, sentence, entity_type="LOC"):
    """
    Generate MCQ using entity as correct answer and dummy distractors
    Note:we will make options using genrator model.
    """
    # Question text
    question_text = generate_question_text(entity_type, "noun", entity_word, sentence)
    
    # Simple distractors (in practice, choose from same entity type and take options from a genrative model)
    distractors = ["خيار1", "خيار2", "خيار3"]
    
    # Combine correct answer + distractors
    options = [entity_word] + distractors
    random.shuffle(options) #?
    
    return {"type": "MCQ", "question": question_text, "options": options, "answer": entity_word}
  

In [24]:
def make_quiz(text):
    questions = []
    
    # Run models
    pos_tags = pos(text)
    ner_tags = ner(text)
    
    # Use NER to create TF + MCQ
    for ner_tag in ner_tags:
        word = ner_tag['word']      # the actual entity, e.g., "أحمد"
        entity = ner_tag['entity']  # the entity label, e.g., "PER"
        
        # Find POS of this word (simple match by string)
        pos_tag = next((p['entity'] for p in pos_tags if p['word'] == word), "noun")
        
        # Generate TF question (pass the required arguments)
        questions.append(generate_TF_question(text, word, entity, ner_tags))
        
        # Generate MCQ question
        questions.append(generate_MCQ_question(word, text, entity_type=entity))
    
    return questions


In [25]:
text = "تأسست جامعة الملك سعود في الرياض عام 1957."
quiz = make_quiz(text)

for q in quiz:
    if q["type"] == "TF":
        print("TF:", q["statement"], "=>", q["answer"])
    elif q["type"] == "MCQ":
        print("MCQ:", q["question"])
        print("Options:", q["options"], "Answer:", q["answer"])


TF: تأسست جامعة الملك سعود في الرياض عام 1957. (صح أم خطأ؟) => True
MCQ: أكمل الفراغ: تأسست ____ الملك سعود في الرياض عام 1957.
Options: ['خيار3', 'خيار2', 'خيار1', 'جامعة'] Answer: جامعة
TF: تأسست جامعة الملك سعود في الرياض عام 1957. (صح أم خطأ؟) => True
MCQ: أكمل الفراغ: تأسست جامعة ____ سعود في الرياض عام 1957.
Options: ['خيار1', 'الملك', 'خيار3', 'خيار2'] Answer: الملك
TF: تأسست جامعة الملك الملك في الرياض عام 1957. (صح أم خطأ؟) => False
MCQ: أكمل الفراغ: تأسست جامعة الملك ____ في الرياض عام 1957.
Options: ['خيار1', 'خيار2', 'خيار3', 'سعود'] Answer: سعود
TF: تأسست جامعة الملك سعود في الرياضX عام 1957. (صح أم خطأ؟) => False
MCQ: أكمل الفراغ: تأسست جامعة الملك سعود في ____ عام 1957.
Options: ['خيار2', 'خيار1', 'الرياض', 'خيار3'] Answer: الرياض



Note:quality of question, quiz  must set question count how will it process long text and randomness of question no all same


In [27]:
def make_flashCards(text, limit=10, window=5):
    """
    Generate flashcards using entity and its definition extracted from context.
    window = how many words before/after entity to include in definition.
    """
    flashcards = []
    
    # Run NER
    ner_tags = ner(text)
    words = text.split()
    
    for ner_tag in ner_tags[:limit]:
        entity = ner_tag['word']
        entity_type = ner_tag['entity']
        
        # Find entity position in words
        try:
            idx = words.index(entity)
        except ValueError:
            idx = 0  # fallback
        
        # Extract a small context window around entity for definition
        start = max(0, idx - window)
        end = min(len(words), idx + window + 1)
        definition_words = words[start:end]
        
        # Remove the entity itself from the definition
        definition = " ".join([w for w in definition_words if w != entity])
        
        flashcards.append({
            "term": entity,
            "definition": definition.strip() or f"نوع: {entity_type}"
        })
    
    return flashcards


In [28]:
text = "إمارة أبوظبي هي إحدى إمارات دولة الإمارات العربية المتحدة السبع"
flashcards = make_flashCards(text, limit=5)

for card in flashcards:
    print(card)


{'term': 'أبوظبي', 'definition': 'إمارة هي إحدى إمارات دولة الإمارات'}
{'term': 'الإمارات', 'definition': 'أبوظبي هي إحدى إمارات دولة العربية المتحدة السبع'}
{'term': 'العربية', 'definition': 'هي إحدى إمارات دولة الإمارات المتحدة السبع'}
{'term': 'المتحدة', 'definition': 'إحدى إمارات دولة الإمارات العربية السبع'}
